## Required Python libraries

In [1]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install lxml

### Download mp3 file

My input:
limit = 5
description = bác sĩ

In [2]:
from podcast_class import Podcast
import dateutil.parser
import requests
import re

def get_episodes_metadata(podcast_items):
    episode_urls = [podcast.find('enclosure')['url'] for podcast in podcast_items]
    episode_titles = [podcast.find('title').text for podcast in podcast_items]
    episode_release_dates = [parse_date(podcast.find('pubDate').text) for podcast in podcast_items]
    return list(zip(episode_urls, episode_titles, episode_release_dates))

def parse_date(date):
	return dateutil.parser.parse(date).strftime('%b-%d-%Y')

def get_mp3_file(url):
	# It redirects the url before you get the actual file
	redirect_url = requests.get(url).url 
	file = requests.get(redirect_url)
	return file

def save_mp3_file(file, file_path):
    with open(file_path, 'wb') as f:
        f.write(file.content)

def simplify_title(title):
	file_name = re.sub(r'[%/&!@#\*\$\?\+\^\\.\\\\]', '', title)[:100]
	file_name1 = file_name.replace('|', '_')
	return file_name1
	#file_name = title.replace('/','-').replace('\\\\','-').replace('.',' ')[:100]

if __name__ == '__main__':
	print("\nPodcasts number u want to downloads(1 - 20): ")
	limit = int(input())
	print("\nWhat type of podcasts do you what to download ? (space, tech, doctor,....)")
	description = str(input())

	print("\n----------------- Downloading podcasts... -----------------\n")

    # This is example, you should change the name and rss file you want to download
    # To download multi:
    # podcast_list = [Podcast('lex-fridman', 'https://lexfridman.com/feed/podcast/'),
    #                 Podcast('name', 'rss_url'),]
	podcast_list = [Podcast('spiderum', 'https://anchor.fm/s/13c13550/podcast/rss')]

	for podcast in podcast_list:
		# download podcast have 'robot' in description
		# you can change what type you what to download
		podcast_items = podcast.search_items(description, limit) 
		episodes_metadata = get_episodes_metadata(podcast_items)
		for episode in episodes_metadata:
			url, title, release_date = episode
			simple_title = simplify_title(title)
			file = get_mp3_file(url)
			# file_path = f'{podcast.download_directory}/{release_date}.mp3'
			file_path = f'{podcast.download_directory}/{simple_title}.mp3'
			save_mp3_file(file, file_path)
			print(file_path, "saved")


Podcasts number u want to downloads(1 - 20): 

What type of podcasts do you what to download ? (space, tech, doctor,....)

----------------- Downloading podcasts... -----------------

./downloads/spiderum/10000 GIỜ - Thời gian bắt buộc để thành chuyên gia _ Nhện tự học _ Bảo Hoàng _ SPIDERUM.mp3 saved
./downloads/spiderum/Nhật kí thằng nhóc học trường Y _ SPIDERUM _ Spidoscared _ Hướng nghiệp.mp3 saved
./downloads/spiderum/13 năm không chiu thua bệnh trầm cảm _ SPIDERUM _ Limitless _ Góc tâm sự.mp3 saved
./downloads/spiderum/LÀM VÌ TIỀN _ SPIDERUM _ HUSKYWANNAFLY _ Cách làm giàu.mp3 saved
./downloads/spiderum/Làm Bác Sĩ khổ thế nào _ SPIDERUM _ DƯƠNG MINH TUẤN _ Hướng nghiệp.mp3 saved


### Transcribe Podcast

In [3]:
import os
import json
import time
import requests
from podcast_class import Podcast

api_key = os.environ['ASSEMLY_AI_KEY'] # Copy the API key on your AssemblyAI account

def create_transcripts(podcast_list, **kwargs):
	all_transcription_metadata = {}
	for podcast in podcast_list:
		podcast_metadata = {}
		downloads = os.listdir(podcast.download_directory)
		for download in downloads:
			print("Uploading", download)
			file_path = f'{podcast.download_directory}/{download}'
			content_url = upload_to_assembly_ai(file_path)
			transcription_id = transcribe_podcast(content_url, **kwargs)
			podcast_metadata[download] = transcription_id

		all_transcription_metadata[podcast.name] = podcast_metadata.copy()

	return all_transcription_metadata

def upload_to_assembly_ai(file_path):
	headers = {'authorization': api_key}
	endpoint = 'https://api.assemblyai.com/v2/upload'
	response = requests.post(endpoint, headers=headers, data=read_file(file_path))
	upload_url = response.json()['upload_url']
	return upload_url

def transcribe_podcast(url, **kwargs):
	headers = {
	    "authorization": api_key,
	    "content-type": "application/json"
	}
	
	json = {'audio_url': url}
	for key, value in kwargs.items():
		json[key] = value
	
	endpoint = 'https://api.assemblyai.com/v2/transcript'
	response = requests.post(endpoint, headers=headers, json=json)
	transcription_id = response.json()['id']
	return transcription_id

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def save_transcription_metadata(metadata, file_path='./transcripts/metadata.json'):
	with open(file_path,'w') as f:
		json.dump(metadata, f)

def load_transcription_metadata(file_path='./transcripts/metadata.json'):
	with open(file_path) as json_file:
		metadata = json.load(json_file)

	return metadata

def save_transcriptions_locally(podcast_list):
	metadata = load_transcription_metadata()
	for podcast in podcast_list:
		podcast_transcriptions = metadata[podcast.name]
		for episode, transcription_id in podcast_transcriptions.items():
			episode_name = os.path.splitext(episode)[0]
			output_path = f'{podcast.transcription_directory}/{episode_name}.txt'
			print('Trying to save', output_path)
			transcription = wait_and_get_assembly_ai_transcript(transcription_id)
			with open(output_path, 'w') as f:
				f.write(transcription['text'])

def get_assembly_ai_transcript(transcription_id):
	headers = {'authorization': api_key}
	endpoint = f'https://api.assemblyai.com/v2/transcript/{transcription_id}'
	response = requests.get(endpoint, headers=headers)
	return response.json()

def wait_and_get_assembly_ai_transcript(transcription_id):
	while True:
		response = get_assembly_ai_transcript(transcription_id)
		if response['status'] == 'completed':
			print("Got transcript")
			break
		elif response['status'] == 'error':
			print("Error getting transcript")
			break
		else:
			print("Transcript not available, wait 5 minutes for transcribing...")
			time.sleep(300) # Try again in 5 minutes

	return response

if __name__ == '__main__':
	print("\n------------------- Transcribing podcasts... -------------------\n")
	#podcast_list = [Podcast('tim-ferriss', 'https://rss.art19.com/tim-ferriss-show')]
	podcast_list = [Podcast('spiderum', 'https://anchor.fm/s/13c13550/podcast/rss')]
    # audio_start_from and audio_end_at: in milliseconds
    # In the below code, the audio start from 1m and end at 5m
	metadata = create_transcripts(podcast_list, audio_start_from=60000, audio_end_at=300000)
	print('Uploaded transcripts')
	save_transcription_metadata(metadata)
	save_transcriptions_locally(podcast_list)


------------------- Transcribing podcasts... -------------------

Uploading 10000 GIỜ - Thời gian bắt buộc để thành chuyên gia _ Nhện tự học _ Bảo Hoàng _ SPIDERUM.mp3
Uploading 13 năm không chiu thua bệnh trầm cảm _ SPIDERUM _ Limitless _ Góc tâm sự.mp3
Uploading Làm Bác Sĩ khổ thế nào _ SPIDERUM _ DƯƠNG MINH TUẤN _ Hướng nghiệp.mp3
Uploading LÀM VÌ TIỀN _ SPIDERUM _ HUSKYWANNAFLY _ Cách làm giàu.mp3
Uploading Nhật kí thằng nhóc học trường Y _ SPIDERUM _ Spidoscared _ Hướng nghiệp.mp3
Uploaded transcripts
Trying to save ./transcripts/spiderum/10000 GIỜ - Thời gian bắt buộc để thành chuyên gia _ Nhện tự học _ Bảo Hoàng _ SPIDERUM.txt
Transcript not available, wait 5 minutes for transcribing...
Got transcript
Trying to save ./transcripts/spiderum/13 năm không chiu thua bệnh trầm cảm _ SPIDERUM _ Limitless _ Góc tâm sự.txt
Got transcript
Trying to save ./transcripts/spiderum/Làm Bác Sĩ khổ thế nào _ SPIDERUM _ DƯƠNG MINH TUẤN _ Hướng nghiệp.txt
Got transcript
Trying to save ./transcript